### **Download Sentiment Dataset**

In [1]:
! pip install -Uq datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
! pip uninstall -y wandb

Found existing installation: wandb 0.19.1
Uninstalling wandb-0.19.1:
  Successfully uninstalled wandb-0.19.1


In [3]:
from datasets import load_dataset

sentiment_dataset = load_dataset("rasyosef/amharic-sentiment")
sentiment_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


README.md:   0%|          | 0.00/1.70k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/269k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/35.2k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/35.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2223 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/279 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/279 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['clean_tweet', 'label'],
        num_rows: 2223
    })
    dev: Dataset({
        features: ['clean_tweet', 'label'],
        num_rows: 279
    })
    test: Dataset({
        features: ['clean_tweet', 'label'],
        num_rows: 279
    })
})

In [4]:
sentiment_dataset["train"].features

{'clean_tweet': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None)}

In [5]:
label_names = sentiment_dataset["train"].features["label"].names
label_names

['negative', 'positive']

### **Tokenize Tweets**

In [6]:
from transformers import AutoTokenizer

model_id = "rasyosef/roberta-medium-amharic"
tokenizer = AutoTokenizer.from_pretrained(model_id)

print(tokenizer.tokenize("ከሀገራቸው ከኢትዮጵያ ከወጡ ግማሽ ምዕተ <mask> ተቆጥሯል።"))

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

['▁ከ', 'ሀገራቸው', '▁ከኢትዮጵያ', '▁ከወጡ', '▁ግማሽ', '▁ምዕተ', ' <mask>', '▁ተቆጥ', 'ሯል።']


In [7]:
def tokenize_dataset(samples):
  tokenized_samples = tokenizer(samples["clean_tweet"], truncation=True, max_length=512)
  return tokenized_samples

tokenize_dataset({"clean_tweet":["ከሀገራቸው ከኢትዮጵያ ከወጡ ግማሽ"]})

{'input_ids': [[0, 17, 18090, 897, 13696, 2476, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [8]:
preprocessed_datasets = sentiment_dataset.map(
    tokenize_dataset,
    batched=True,
    remove_columns=["clean_tweet"]
  )
preprocessed_datasets

Map:   0%|          | 0/2223 [00:00<?, ? examples/s]

Map:   0%|          | 0/279 [00:00<?, ? examples/s]

Map:   0%|          | 0/279 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 2223
    })
    dev: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 279
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 279
    })
})

### **Load Amharic BERT model**

In [9]:
id2label={i: label for i, label in enumerate(label_names)}
label2id={label: i for i, label in enumerate(label_names)}

print(id2label, label2id)

{0: 'negative', 1: 'positive'} {'negative': 0, 'positive': 1}


In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id,
    # device_map="cuda"
)

config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/169M [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at rasyosef/roberta-medium-amharic and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### **Finetuning**

In [11]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

In [12]:
from transformers import TrainingArguments

batch_size = 32
epochs = 2

training_args = TrainingArguments(
    output_dir=model_id+"-finetuned",
    learning_rate=3e-5,
    # lr_scheduler_type="linear",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    # metric_for_best_model="f1",
    fp16=True,
    seed=32,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
  metric1 = evaluate.load("accuracy")
  metric2 = evaluate.load("precision")
  metric3 = evaluate.load("recall")
  metric4 = evaluate.load("f1")

  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)

  accuracy = metric1.compute(predictions=predictions, references=labels)["accuracy"]
  # precision = metric2.compute(predictions=predictions, references=labels, average='macro')["precision"]
  # recall = metric3.compute(predictions=predictions, references=labels, average='macro')["recall"]
  # f1 = metric4.compute(predictions=predictions, references=labels, average='macro')["f1"]

  return {
      "accuracy": accuracy,
      # "precision": precision,
      # "recall": recall,
      # "f1": f1
  }

compute_metrics((np.array([[1,0], [0,1]]), np.array([0,1])))

{'accuracy': 1.0}

In [14]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=preprocessed_datasets["train"],
    eval_dataset=preprocessed_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-14-b0b54413edf8>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
trainer.evaluate()

{'eval_loss': 0.6896701455116272,
 'eval_model_preparation_time': 0.0024,
 'eval_accuracy': 0.5519713261648745,
 'eval_runtime': 3.0617,
 'eval_samples_per_second': 91.126,
 'eval_steps_per_second': 2.94}

In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy
1,0.501700,0.365584,0.002400,0.853047
2,0.334900,0.337318,0.002400,0.863799


TrainOutput(global_step=140, training_loss=0.41831178665161134, metrics={'train_runtime': 23.9234, 'train_samples_per_second': 185.843, 'train_steps_per_second': 5.852, 'total_flos': 46666964592276.0, 'train_loss': 0.41831178665161134, 'epoch': 2.0})

In [17]:
trainer.evaluate()

{'eval_loss': 0.3373182713985443,
 'eval_model_preparation_time': 0.0024,
 'eval_accuracy': 0.8637992831541219,
 'eval_runtime': 2.0429,
 'eval_samples_per_second': 136.568,
 'eval_steps_per_second': 4.405,
 'epoch': 2.0}

### **Evaluate Model**

In [18]:
import torch

num_samples = len(preprocessed_datasets["test"])
inputs = data_collator([preprocessed_datasets["test"][i] for i in range(num_samples)])
# print(inputs)

y_pred = []

#output = model(**inputs)
for i in range(0, len(inputs["input_ids"]), 8):
  output = model(**{k: v[i:i+8].to("cuda") for k, v in inputs.items()})
  y_pred.extend(np.argmax(output.logits.tolist(), axis=-1))

y_test = np.array(preprocessed_datasets["test"]["label"])

print(y_pred)
print(y_test)

[1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
[1 1 0 1 0 0 1 1 0 0 1 0 1 0 1 1 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1
 0 1 1 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1 1 0 0 1 0
 0 1 0 1 1 0

In [19]:
from sklearn import metrics

metrics.confusion_matrix(y_test, y_pred)

array([[128,  22],
       [ 22, 107]])

In [20]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85       150
           1       0.83      0.83      0.83       129

    accuracy                           0.84       279
   macro avg       0.84      0.84      0.84       279
weighted avg       0.84      0.84      0.84       279



### **Test in Pipeline**

In [21]:
from transformers import pipeline

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)
pipe("ቆንጆ ፊልም ነው")

Device set to use cuda:0


[{'label': 'positive', 'score': 0.9719126224517822}]

In [22]:
pipe("አሪፍ ልብስ")

[{'label': 'positive', 'score': 0.9652831554412842}]

In [23]:
pipe("ጅላንፎ")

[{'label': 'negative', 'score': 0.9317735433578491}]

In [24]:
pipe("እዚህ ሰፈር በማታ መጓዝ እፈራለሁ")

[{'label': 'negative', 'score': 0.7667742967605591}]

In [25]:
pipe("ዩክሬን እና ሩስያ ከባድ ውግያ ላይ ናቸው")

[{'label': 'negative', 'score': 0.8986777067184448}]

**Mis-classified**

In [26]:
misclassified = []
for i in range(len(y_pred)):
  if y_pred[i] != y_test[i]:
    misclassified.append(i)

print(misclassified)

[2, 8, 12, 13, 23, 25, 29, 33, 49, 53, 54, 59, 63, 65, 72, 74, 81, 89, 99, 100, 103, 107, 115, 120, 125, 144, 146, 147, 173, 201, 203, 210, 211, 219, 220, 239, 240, 243, 249, 250, 258, 261, 269, 270]


In [27]:
for idx in misclassified:
  input_ids = preprocessed_datasets["test"][idx]["input_ids"]
  label = preprocessed_datasets["test"][idx]["label"]
  print(label, tokenizer.decode(input_ids))

0 <s> : ለማ መገርሳ ሆይ! የቀድሞ ወዳጅህ ናሁሰናይ ከመቐለ ይህን ባለ 6 ነጥብ መልእክት ልኮልሃል፦ 1. ጠባቂዎችህን ሙሉ በሙሉ ከአብይ ንኪኪ እና ትእዛዝ አፅዳ፡፡ የጥበቃ ጥንካሬው ሶስት እጥፍ ጨምር፡...</s>
0 <s> እንደሱማ ከሆነ እናንተም ቦታ ላይኖራችሁ ነው</s>
1 <s> እንዲህ አይነት ርዕስ ጽፎ ለበዐል ይጠብቁን ድንግርግር ይላል ግን አሻም በበዓልም አልቀልድም ካለ ምን ይደረግ? 😃 ስሜነህ አያሌውና ሱራፌል ሲጠይቁ እንኳን ብንሰማ ይሻለናል በውነት።</s>
0 <s> ከእንደ አንተ አይነቱ ደደብ መሀይም ጭራቅ የምንላቀቅበት ዘመን ይሁን!!! ሁሌም አብየየየየ</s>
0 <s> ሰሊ እውነት አንቺ አማኝ ነሽ ከዚህ ሂጂ😄</s>
0 <s> ልክ ብለሃል። እኛማ ተተብትበናል።</s>
0 <s> ያሰብከው ሁሉ ይሳካ! በእውነትና እውቀትና በተመሰረትርው ራዕይ ፓርቲ ኢትዮጵያና ህዝቦችዋ ታፍረውና ተከውረው ይኖራሉ።</s>
0 <s> አጋንንቶች ነፍሴን ይመገባሉ። መሪዬን አገለግላለሁ ፡፡ እኔ ነፍሳትን አመጣለሁ</s>
0 <s> በመካከላችን ማንም የለም ፣ በገዛ እጄ ከገደለው ፍጥረት ሁሉ ጋር እኖራለሁ ምንም እንኳን ከእንግዲህ ከእኛ ጋር ባይሆንም</s>
0 <s> ታሪካችን ፡ እንደገና ፡ ተበዉዞ ፡ መስራት ፡ አለበት ፡ እስካሁን ፡ ያለው ፡ የኣንድ ፡ ወገን ፡ ታሪክ ፡ ነው።</s>
0 <s> : በተስፋ የምትጠብቅ እናት በሠላም የላከችው ልጇ ሬሳው በሳጥን ተጭኖ ሲመጣላት የሚሰማትን የሀዘን ስሜት ከባድ ነው ስለዚህ እማራለሁ ብለሁ ብለው ሄደው እሬሳቸው ለቤተ...</s>
0 <s> ከሀገሬው ይልቅ ለቻይናዉያን ምቹ የሆነች ሀገር <unk>...</s>
0 <s> ተዋ😄 እኔ ለሃገሪቷ ስለትምህት ፣ ሰላም፣ ጤና ፣ መሰረተ ልማት ፣ ውጭ ጉዳይ ወዘተ... ይሄን ፖሊሲ ይዤ 